In [1]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/nikhilw/Documents/Devel/PyProj_SQLAgent/simple_db_QnA/App


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/Sample_DB.db")
db._sample_rows_in_table_info = False
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT name FROM sqlite_master LIMIT 10;")
db.run("SELECT * FROM Drugs LIMIT 10;")

sqlite
['Cost_Of_Care', 'Diseases', 'Diseases_By_Channel', 'Drugs', 'Member_Costs', 'Members', 'Pharmacies']


"[('Curr Rank', 'Prev Rank', 'Drug Name', 'Category', 'Utilizers', 'Claims', 'Plan Cost', '% of Total Plan Cost', 'Plan Cost PMPM', 'Plan Cost/Claim*', 'Member Cost/Claim', 'Member Cost Share', 'Prev Utilizers', 'Prev Claims', 'Prev Plan Cost', 'Claims PMPM Trend*', 'Plan Cost/Claim Trend*', 'Plan Cost PMPM Trend', 'Plan Cost Trend', 'Srx'), (1, 18, 'MOUNJARO', 'Diabetes', 7812, 54218, 51838552.35, 0.062, 9.16, 900.13, 33.83, 0.034, 5216, 7080, 6296048.11, 6.073, 0.042, 6.371, 7.234, 'N'), (2, 1, 'HUMIRA', 'Inflammatory Disorder', 899, 7057, 50498194.86, 0.06, 8.92, 7148.67, 927.4, 0.115, 920, 7150, 45587843.11, -0.116, 0.121, -0.008, 0.108, 'Y'), (3, 3, 'STELARA', 'Inflammatory Disorder', 276, 1188, 23055438.25, 0.027, 4.07, 10829.23, 2145.43, 0.1, 251, 1118, 21234652.16, -0.068, 0.043, -0.028, 0.086, 'Y'), (4, 4, 'TRULICITY', 'Diabetes', 3533, 19569, 20053132.6, 0.024, 3.54, 808.72, 41.56, 0.039, 3859, 19037, 20644961.04, -0.163, 0.038, -0.13, -0.029, 'N'), (5, 5, 'BIKTARVY', 'HIV In

In [44]:
db.get_usable_table_names()
db.get_table_info(["Drugs","Diseases"])

'\nCREATE TABLE "Diseases" (\n\t"Curr_Rank" INTEGER, \n\t"Prev_Rank" INTEGER, \n\t"Disease" VARCHAR(100), \n\t"Utilizers" INTEGER, \n\t"Claims" INTEGER, \n\t"Plan_Cost" DECIMAL(18, 2), \n\t"Percent_of_Total_Plan_Cost" DECIMAL(5, 2), \n\t"Plan_Cost_PMPM" DECIMAL(10, 2), \n\t"Plan_Cost_Per_Claim" DECIMAL(12, 2), \n\t"Member_Cost_Share" DECIMAL(5, 2), \n\t"Prev_Utilizers" INTEGER, \n\t"Prev_Claims" INTEGER, \n\t"Prev_Plan_Cost" DECIMAL(18, 2), \n\t"Plan_Cost_Trend" DECIMAL(6, 2), \n\t"Claims_PMPM_Trend" DECIMAL(6, 2), \n\t"Plan_Cost_Per_Claim_Trend" DECIMAL(6, 2), \n\t"Plan_Cost_PMPM_Trend" DECIMAL(6, 2), \n\tPRIMARY KEY ("Disease")\n)\n\n\nCREATE TABLE "Drugs" (\n\t"Curr_Rank" INTEGER, \n\t"Prev_Rank" INTEGER, \n\t"Drug_Name" VARCHAR(100), \n\t"Category" VARCHAR(100), \n\t"Utilizers" INTEGER, \n\t"Claims" INTEGER, \n\t"Plan_Cost" DECIMAL(18, 2), \n\t"Percent_of_Total_Plan_Cost" DECIMAL(5, 2), \n\t"Plan_Cost_PMPM" DECIMAL(10, 2), \n\t"Plan_Cost_Per_Claim" DECIMAL(12, 2), \n\t"Member_Cost_

In [45]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
	os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

try:
	response = llm.invoke("What is the capital of India?")
	print("Response:", response)
except Exception as e:
	print("Error connecting to OPENAI API:", e)
	print("Please check your internet connection, firewall settings, and API key.")
print("llm:", llm)


Response: content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be77oZ9XrPnd17RYZPojOTvvO9mBD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--94799fcb-8f57-46a3-88c2-20eaa63c59c8-0' usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
llm: client=<openai.resources.chat.completions.completions.Completions object at 0x118d2d660> async_client=<openai.resources.chat.completions.completions.AsyncCompletions ob

In [46]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str # User's question to be answered.
    query: str # SQL query generated from the question.
    result: str # Result of the query execution in json.
    answer: str  # Answer to the question based on the query result.
    columns: list[str]  # List of column names in the result.
    followup: list[str] # List of followup questions 

In [47]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
Output only 1 query even for multiple questions. 
All decimal numbers in the output should be formatted to only 2 digits after the decimal.
Do not add any explanation. Do not add any comments.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
Output only 1 query even for multiple questions. 
All decimal numbers in the output should be formatted to only 2 digits after the decimal.
Do not add any explanatio

In [48]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    print("Prompt:", prompt)
    # print("Prompt:", prompt.messages[1].content)

    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [49]:
db.get_table_info(["Drugs","Diseases"])

'\nCREATE TABLE "Diseases" (\n\t"Curr_Rank" INTEGER, \n\t"Prev_Rank" INTEGER, \n\t"Disease" VARCHAR(100), \n\t"Utilizers" INTEGER, \n\t"Claims" INTEGER, \n\t"Plan_Cost" DECIMAL(18, 2), \n\t"Percent_of_Total_Plan_Cost" DECIMAL(5, 2), \n\t"Plan_Cost_PMPM" DECIMAL(10, 2), \n\t"Plan_Cost_Per_Claim" DECIMAL(12, 2), \n\t"Member_Cost_Share" DECIMAL(5, 2), \n\t"Prev_Utilizers" INTEGER, \n\t"Prev_Claims" INTEGER, \n\t"Prev_Plan_Cost" DECIMAL(18, 2), \n\t"Plan_Cost_Trend" DECIMAL(6, 2), \n\t"Claims_PMPM_Trend" DECIMAL(6, 2), \n\t"Plan_Cost_Per_Claim_Trend" DECIMAL(6, 2), \n\t"Plan_Cost_PMPM_Trend" DECIMAL(6, 2), \n\tPRIMARY KEY ("Disease")\n)\n\n\nCREATE TABLE "Drugs" (\n\t"Curr_Rank" INTEGER, \n\t"Prev_Rank" INTEGER, \n\t"Drug_Name" VARCHAR(100), \n\t"Category" VARCHAR(100), \n\t"Utilizers" INTEGER, \n\t"Claims" INTEGER, \n\t"Plan_Cost" DECIMAL(18, 2), \n\t"Percent_of_Total_Plan_Cost" DECIMAL(5, 2), \n\t"Plan_Cost_PMPM" DECIMAL(10, 2), \n\t"Plan_Cost_Per_Claim" DECIMAL(12, 2), \n\t"Member_Cost_

In [50]:
write_query({"question": "Whats the Total Invoice for each month?"})

Prompt: messages=[SystemMessage(content='\nGiven an input question, create a syntactically correct sqlite query to\nrun to help find the answer. Unless the user specifies in his question a\nspecific number of examples they wish to obtain, always limit your query to\nat most 10 results. You can order the results by a relevant column to\nreturn the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the\nfew relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema\ndescription. Be careful to not query for columns that do not exist. Also,\npay attention to which column is in which table.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\nOutput only 1 query even for multiple questions. \nAll decimal numbers in the output should be formatted to only 2 digits after the decimal.\nDo not add any explanation. Do not add any comments.\n\nO

{'query': "SELECT strftime('%Y-%m', Member_DOB) AS Month, SUM(Total_Cost) AS Total_Invoice \nFROM Members \nJOIN Cost_Of_Care ON Members.Member_ID = Cost_Of_Care.Member_ID \nGROUP BY Month \nORDER BY Month \nLIMIT 10;"}

In [51]:
from typing_extensions import Annotated

class ColumnOutput(TypedDict):
    columns: Annotated[list[str], "List of column names in the SELECT statement, in order."]

def extract_columns(state: State):
    """Use LLM to extract column names from the SQL query."""
    prompt = (
        "Given the following SQL query, list the column names that will appear in the result set, in order, as a Python list of strings.\n\n"
        f"SQL Query:\n{state['query']}\n\n"
        "Columns:"
    )
    structured_llm = llm.with_structured_output(ColumnOutput)
    result = structured_llm.invoke(prompt)
    return {"columns": result["columns"]}

In [52]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    str_result = execute_query_tool.invoke(state["query"])
    # print("Query:", state["query"])
    return {"result": str_result}

In [53]:
from typing_extensions import Annotated

class QuestionList(TypedDict):
    questions: Annotated[list[str], "List of followup questions."]

def FollowupQuestions(state: State):
    """Use LLM to extract column names from the SQL query."""
    prompt = (
        "Given the following SQL query and Result, list the next 3 followup questions that might be relevant for the user.\n\n"
        f"SQL Query:\n{state['query']}\n\n"
        f"Result:\n{state['result']}\n\n"
        "Follow-up Questions:"
    )
    structured_llm = llm.with_structured_output(QuestionList)
    print("Prompt:", prompt)
    result = structured_llm.invoke(prompt)
    return {"Followup_Questions": result["questions"]}

In [54]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'

    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [55]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, extract_columns, execute_query, generate_answer, FollowupQuestions]  # Insert extract_columns after write_query
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [63]:
str_query = ""
str_result = ""
str_answer = ""
str_columns = []
for step in graph.stream(
    {"question": "Which are the most prevalent diseases among members of Plan Type A?"}, stream_mode="updates"
#    {"question": "Can we see a breakdown of Aerosmith's revenue by album or track for each of those years to identify their top-performing content? ?"}, stream_mode="updates"
):
    print("Step:", step)
    if "write_query" in step:
        str_query = step["write_query"]["query"]
    if "execute_query" in step:
        str_result = step["execute_query"]["result"]
    if "generate_answer" in step:
        str_answer = step["generate_answer"]["answer"]
    if "extract_columns" in step:
        str_columns = step["extract_columns"]["columns"]
    if "Followup_Questions" in step:
        followup_questions = step["FollowupQuestions"]["Followup_Questions"]
    print("Query :", str_query)





Prompt: messages=[SystemMessage(content='\nGiven an input question, create a syntactically correct sqlite query to\nrun to help find the answer. Unless the user specifies in his question a\nspecific number of examples they wish to obtain, always limit your query to\nat most 10 results. You can order the results by a relevant column to\nreturn the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the\nfew relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema\ndescription. Be careful to not query for columns that do not exist. Also,\npay attention to which column is in which table.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\nOutput only 1 query even for multiple questions. \nAll decimal numbers in the output should be formatted to only 2 digits after the decimal.\nDo not add any explanation. Do not add any comments.\n\nO